## 方法默认值和命名参数列表
摘抄自《scala程序设计（第2版）》————2.5.1方法默认值和命名参数列表

以下是修改后的 Point case 类:
```scala
// src/main/scala/progscala2/typelessdomore/shapes/shapes.scala
package progscala2.typelessdomore.shapes

case class Point(x: Double = 0.0, y: Double = 0.0) {         // ➊

  def shift(deltax: Double = 0.0, deltay: Double = 0.0) =    // ➋
    copy (x + deltax, y + deltay)
}
```

➊ 如同前文,定义 Point 类,并提供默认的初始化值。  
➋ 新的 shift 方法,用于从现有的 Point 对象中对“点”进行平移,从而创建一个新的Point 对象。它使用了 copy 方法, copy 方法也是 case 类自动创建的  

**copy 方法允许你在创建 case 类的新实例时,只给出与原对象不同部分的参数,这一点对于大一些的 case 类非常有用**:
```scala
scala> val p1 = new Point(x = 3.3, y = 4.4)   // 显式使用命名参数列表。
p1: Point = Point(3.3,4.4)

scala> val p2 = p1.copy(y = 6.6)             // 指定新的y值,创建新实例。
p2: Point = Point(3.3,6.6)
```

**命名参数列表让客户端代码更具可读性。当参数列表很长,且有若干参数是同一类型时,bug 容易避免,因为在这种情况下很容易搞错参数传入的顺序。当然,更好的做法是一开始就避免出现过长的参数列表。**

## 方法具有多个参数列表
摘抄自《scala程序设计（第2版）》————2.5.2方法具有多个参数列表

接下来,我们对 Shape 类进行修改,特别是其中的 draw 方法:
```scala
abstract class Shape() {
  /**
   * Draw takes TWO argument LISTS, one list with an offset for drawing,
   * and the other list that is the function argument we used previously.
   */
  /**
   * draw 带两个参数列表,其中一个参数列表带着一个表示绘制偏移量的参数
   * 另一个参数列表是我们之前用过的函数参数。
   */
  def draw(offset: Point = Point(0.0, 0.0))(f: String => Unit): Unit =
    f(s"draw(offset = $offset), ${this.toString}")
}

case class Circle(center: Point, radius: Double) extends Shape

case class Rectangle(lowerLeft: Point, height: Double, width: Double) extends Shape
```

没错,这里的 draw 方法有两个参数列表,每个参数列表都有一个参数,而不是拥有一个具有两个参数的参数列表。第一个参数列表允许你指定 Point 对象的偏移量,供绘制使用。默认值 Point(0.0, 0.0) ,表示没有偏移。第二个参数列表与之前的 draw 函数相同,其中的参数是用来绘制所用的函数的。

**你可以任意指定参数列表的个数,但实际上很少有人使用两个以上的参数列表。**

那么,**为什么要允许多个参数列表呢?当最后一个参数列表只包含一个表示函数的参数时,多个参数列表的形式拥有整齐的块结构语法。**以下是我们调用新的 draw 方法的表达方式:
```scala
s.draw(Point(1.0, 2.0))(str => println(s"ShapesDrawingActor: $str"))
```

Scala 允许我们把参数列表两边的圆括号替换为花括号,因此,这一行代码还可以写为:
```scala
s.draw(Point(1.0, 2.0)){str => println(s"ShapesDrawingActor: $str")}
```

如果函数字面量不能在一行内完成,我们可以重写为以下方式:
```scala
s.draw(Point(1.0, 2.0)) { str =>
    println(s"ShapesDrawingActor: $str")
}
```

或写为等价的形式:
```scala
s.draw(Point(1.0, 2.0)) {
    str => println(s"ShapesDrawingActor: $str")
}
```

这一写法很像我们之前常用来写 if 和 for 表达式或方法体的代码块。只不过,在这里的{...} 块所表示的函数是我们要传递给 draw 方法的参数。

**当函数字面量很长时,这种用 {...} 代替 (...) 的“语法糖”使得代码看起来美观多了。此时的代码更像我们所熟悉和喜爱的块结构语法。**

如果我们使用缺省的偏移量,第一个圆括号就不能省略:
```scala
s.draw(){str => println(s"ShapesDrawingActor: $str")}
```

如同 Java 方法一样, draw 方法也可以只使用一个带两个参数值的参数列表。如果那样,客户端代码就会像这样写:
```scala
s.draw(Point(1.0, 2.0),
    str => println(s"ShapesDrawingActor: $str")
)
```

这份代码并没那么清晰和优雅。使用默认值开启 offset 也没那么便捷,因此我们不得不对参数进行命名:
```scala
s.draw(f = str => println(s"ShapesDrawingActor: $str"))
```

**第二个优势是在之后的参数列表中进行类型推断。**如以下例子:
```scala
scala> def m1[A](a: A, f: A => String) = f(a)
m1: [A](a: A, f: A => String)String

scala> def m2[A](a: A)(f: A => String) = f(a)
m2: [A](a: A)(f: A => String)String

scala> m1(100, i => s"$i + $i")
<console>:12: error: missing parameter type
m1(100, i => s"$i + $i")
^

//必须加上参数类型才行
scala> m1(100, (i:Int) => s"$i + $i")
res0: String = 100 + 100

scala> m2(100)(i => s"$i + $i")
res1: String = 100 + 100
```

函数 m1 和函数 m2 看起来几乎一模一样,但我们需要注意用相同的参数调用它们时 m1 和m2 的表现。我们传入 Int 和一个函数 Int => String ,对于 m1 ,Scala 无法推断该函数的参数 i , m2 则可以。

**使用多个参数列表的第三个优势是,我们可以用最后一个参数列表来推断隐含参数。隐含参数是用 implicit 关键字声明的参数。**当相应方法被调用时,我们可以显式指定这个参数,或者也可以不指定,这时编译器会在当前作用域中找到一个合适的值作为参数。隐含参数可以代替参数默认值,而且更加灵活。我们这就来研究一个 Scala 库中使用隐含参数的例子 Future 。

## 嵌套方法的定义与递归
摘抄自《scala程序设计（第2版）》————2.5.4嵌套方法的定义与递归

方法的定义还可以嵌套。**当你将一个很长的方法重构为几个更小的方法时,如果这些小的辅助方法只在该方法中调用,就可以用嵌套方法。我们将这些辅助函数嵌套定义在原方法中,它们便对其他外层的代码不可见,包括类中的其他方法。**

以下代码实现了阶乘的计算,在这个方法中,我们调用了另一个嵌套的方法去完成阶乘的实际计算:
```scala
// src/main/scala/progscala2/typelessdomore/factorial.sc

def factorial(i: Int): Long = {
  def fact(i: Int, accumulator: Long): Long = {
    if (i <= 1) accumulator
    else fact(i - 1, i * accumulator)
  }
    
  fact(i, 1L)
}

(0 to 5) foreach ( i => println(factorial(i)) )
```
以下为代码运行的输出:
```
1
1
2
6
24
120
```

辅助函数递归地调用它本身,并传入一个 accumulator 参数,阶乘的计算结果保存在该参数中。注意,当计数器 i 达到 1 时,我们就将阶乘的计算结果返回。(这里我们不考虑负整数参数,负整数的输入是无效的,本函数在 i <= 1 时返回 1。)定义好嵌套的方法后,factorial 调用该方法,传入参数 i ,并累计乘法的初始值 1。

**很容易忘记调用嵌套的函数!如果编译器提示,能找到 Unit 但找不到 Long,可能就是因为你忘记调用嵌套函数了。**

是否注意到,我们两次用 i 作为参数名?第一次是 factorial 方法的参数,第二次是嵌套的 fact 方法的参数。在 fact 方法中使用的 i 参数“屏蔽”了外部 factorial 方法的 i 参数。这样做是允许的,因为我们在 fact 方法中并不需要外部的 i ,我们只在 factorial 结尾调用 fact 的时候才需要它。

类似方法中声明的局部变量,嵌套的方法也只在声明它的方法中可见。

观察这两个方法的返回值。因为阶乘的计算结果增长非常快,我们选择使用 Long 类型,而不使用 Scala 自动推断的 Int 类型。如果使用 Int 类型, factorial 就不需要上述的类型注释了。然而,**我们必须要为 fact 声明返回类型。因为这是一个递归方法,Scala 采用的是
局部作用域类型推断,无法推断出递归函数的返回类型。**

对递归函数你也许会感到一丝不安。我们是否在冒风险? JVM 和许多其他语言环境并不对尾递归做优化,否则尾递归会将递归转为循环,可以避免栈溢出。(尾递归一词,表示调用递归函数是该函数中最后一个表达式,该表达式的返回值就是所调用的递归函数的返回值。)

**递归是函数式编程的特点,也是优雅地实现很多算法的强大工具。所以,Scala 编译器对尾递归做了有限的优化。它会对函数调用自身做优化,但不会优化所谓的 trampoline 的情况,也就是“a 调用 b 调用 a 调用 b”的情形。**

你可能仍然想知道自己写的尾递归是否正确,编译器是否对自己的尾递归执行了优化。没有人希望在生产环境中出现栈空间崩溃。幸运的是,如果你**加一个 tailrec 关键字,编译器会告诉你代码是否正确地实现了尾递归**,如以下 factorial 的改良版本:
```scala
// src/main/scala/progscala2/typelessdomore/factorial-tailrec.sc
import scala.annotation.tailrec

def factorial(i: Int): Long = {
  @tailrec
  def fact(i: Int, accumulator: Int): Long = {
    if (i <= 1) accumulator
    else fact(i - 1, i * accumulator)
  }

  fact(i, 1)
}

(0 to 5) foreach ( i => println(factorial(i)) )
```

如果 fact 不是尾递归,编译器就会抛出错误。我们用这个特性在 REPL 中写出递归的Fibonacci 函数:
```scala
import scala.annotation.tailrec

@tailrec
def fibonacci(i: Int): Long = {
  if (i <= 1) 1L
  else fibonacci(i - 2) + fibonacci(i - 1)
}
```
```
error: could not optimize @tailrec annotated method fibonacci: it contains a recursive call not in tail position
  else fibonacci(i - 2) + fibonacci(i - 1)
```
我们有两个递归调用,然后又对调用的结果做计算,而不是只在结尾调用一次递归函数,因此这个函数不是尾递归的。

## Scala可变参数
Scala 允许你指明函数的最后一个参数可以是重复的。这可以允许客户向函数传入可变长度参数列表。想要标注一个可变参数，在参数的类型之后放一个星号。例如：
```scala
scala> def echo(args: String*) = for (arg <- args) println(arg)
echo: (String*)Unit
```

这样定义， echo 可以被零个至多个 String 参数调用：
```scala
scala> echo()
scala> echo("one")
one
scala> echo("hello", "world!")
hello
world!
```

函数内部，重复参数的类型是声明参数类型的数组。因此， echo 函数里被声明为类型“ String* ”的 args 的类型实际上是 Array[String] 。然而，如果你有一个合适类型的数组，并尝试把它当作重复参数传入，你会得到一个编译器错误：
```scala
scala> val arr = Array("What's", "up", "doc?")
arr: Array[java.lang.String] = Array(What's, up, doc?)
scala> echo(arr)
<console>:7: error: type mismatch;
 
found : Array[java.lang.String]
required: String
```

要实现这个做法，你需要在数组参数后**添加一个冒号和一个 _* 符号**，像这样：
```scala
scala> echo(arr: _*)
What's
up
doc?
```
这个标注告诉编译器把 arr 的每个元素当作参数，而不是当作单一的参数传给 echo 。因此**当形参为String*时，不能直接把类型为Array[String]的实参直接传入，需要通过:_*进行转换。这个语法对List等也是有效的，这个和Python的位置参数传参形式有点类似。**

java中使用的可变参数用到 ...  注意区分
```java
public void echo(String... args){
    for(String arg:args)
        System.out.println("hello "+arg);
}
```

## 偏(部分)应用函数
**第一种方式：**

考虑如下带两个参数列表的简单方法:
```scala
scala> def cat1(s1: String)(s2: String) = s1 + s2
cat1: (s1: String)(s2: String)String
```
如果我们需要一个专门的版本,要求第一个字符串总是 Hello,我们可以通过偏应用函数来定义这样的函数:
```scala
scala> val hello = cat1("Hello ") _
hello: String => String = <function1>

scala> hello("World!")
res0: String = Hello World!

scala> cat1("Hello ")("World!")
res1: String = Hello World!
```
REPL 的输出表明, hello 是一个 <function1> ,也就是带一个参数的函数。
    
我 们 调 用 cat1 时 给 出 了 第 一 个 参 数 列 表, 后 面 跟 上 一 个 下 划 线( _ ), 用 它 来 定 义 了hello 。

关键就在于偏应用函数。对于拥有多个参数列表的函数而言,如果你希望忽略其中一个或多个参数列表,可以通过定义一个新函数来实现。也就是说,你给出了部分所需的参数。为了避免潜在的表达式歧义,Scala 要求在后面加上下划线,用来告诉编译器你的真实目的。注意,这个特性只对函数的多个参数列表有效,对一个参数列表中的多个参数的情况并不适用。


**第二种方式：**

部分应用表示一个函数的调用并不需要全部的参数，可以使用 “_” 这个特殊符号代表一个参数，使用部分应用之后函数的返回值是函数，而不是具体的值。
```scala
def add(x: Int, y: Int) = x + y

val addOne = add(1, _: Int)

addOne(4) // 5
addOne(6) // 7
```